# Обучение на графах (задача)

Главная проблема - разыскать такое представление графа, которое отобразит графовую структуру в модель машинного обучения. Решается это так:
- графовые стаистики (коэф.кластеризации и т.п. и т.д.)
- ядерные методы [Graph Kernels 2010](https://www.jmlr.org/papers/volume11/vishwanathan10a/vishwanathan10a.pdf)
- манипуляции с признаками для оценки локальной близости
- обучение представлению графа

Последняя идея носит название representation learning и позволяет отобразить структурную информацию о графе в виде векторов.

Представление информации в нодах или субграфах в виде векторов - проблема. Оказывается, как правило, данные содержатся в многомерном неевклидовом пространстве. А нам нужны векторы в пространстве, в котором мы можем классифицировать, кластеризировать, предсказывать какую-то вероятность и делать кучу других манипуляций ML. Соответственно **задача - перегнать многомерный вектор признаков в низкоразмерный, размерностью $d$, желательно из $\mathbb{R}$**.

![захарьевские против солнцевских](img/01.png)

Тот самый Zachary Karate Club social network, слева показана связь между группировской захарьевских, солнцевских и других бандитов, а справа двумерное представление, позволяющее как-то работать с этой сетью осмысленно с точки зрения ML.

В статье:
**[Representation Learning on Graphs: Methods and Applications](https://arxiv.org/abs/1709.05584)**
William L. Hamilton
Rex Ying
Jure Leskovec
arXiv:1709.05584v3 \[cs.SI\] 10 Apr 2018
, если опустить подробности, задача эта описывается так:

Если мы имеем граф $G = (V, E)$ и ассоциированные с ним матрицу смежности $A$ и матрицу $X$, содержащую атрибуты нод, так что $X \in \mathbb{R}^{m \times \vert V \vert}$, то задачей является получение векторов $z \in \mathbb{R}^d$ для каждой ноды, таких, что $d \ll \vert V \vert$.

# Модель

- функция попарной похожести (pairwise similarity function) $S_g : V \times V \rightarrow \mathbb{R}^+$, определенная над графом $G$. Функция измеряет схожесть между нодами.
- энкодер, генерирующий эмбеддинги $ENC: V \rightarrow  \mathbb{R}^d$
- декодер, реконструирующий статистики графа из эмбеддингов $DEC : \mathbb{R}^d \times \mathbb{R}^d \rightarrow \mathbb{R}^+$ \*
- функция потерь (специфична для каждой задачи) - оценивает несоответствие между декодированным (оцененным) значением близости $DEC(\mathbf{z}_i, \mathbf{z}_j)$ и истинным $S_g(v_i, v_j)$

\* для двух нод в графе $DEC(ENC(v_i), ENC(v_j)) = DEC(\mathbf{z}_i, \mathbf{z}_j) \approx S_g(v_i, v_j)$ декодер восстанавливает "попарную близость" нод в графе из эмбеддингов

# Энкодеры

**Самый простой метод** - shallow encoding, для которого мы можем определить энкодер как функцию: $ENC(v_i) = Z v_i$, где $Z \in \mathbb{R}^{m \times \vert V \vert}$.

![шаловливый энкодер](img/02.png)

К подобным методам относятся различные <span class="burk">матричные факторизации</span> (graph factorisation, GraRep, HOPE и т.д.) и RandomWalk модели (DeepWalk, node2wec).

![шаловливые энкодеры - методы](img/03.png)

**Проблемы подхода**

- никакие параметры не используются совместно в энкодере 
- <span class="burk">число параметров растет как функция от количества нод</span> $O(|V|)$
- подход не учитывает атрибуты нод, которые в большинстве случаев содержат важную информацию о графе
- подход генерирует эмбеддинги только для данных, которые есть среди обучаемых. Модель проблематично обобщить на данные, которые модель никогда не видела

## Generalized encoder-decoder architectures

**(1) Neighborhood autoencoder methods** (<span class="burk">автоэнкодеры окрестностей</span>)

- DNGR (deep neural graph representation)
- SDNE (structural deep network embeddings)

Каждая нода $v_i$ ассоциируется с высокаразмерным "вектором близости" $S_i \in \mathbb{R}^{\vert V \vert}$, содержащем информацию о близости ноды кнодам в окресности в графе. Затем данный вектор сжимается до нужной размерности.

![Generalized encoder-decoder architectures](img/06.png)

Проблема такого подхода в том, что он невероятно дорогой. К тому же метод статичный и плохо работает с изменяющимися графами.

**(2) Neighborhood aggregation and convolutional encoders**

Этот метод позволяет <span class="burk">агрегировать</span> «месседжи» от соседей ноды, которые, в свою очередь, базируются на «месседжах», агрегированных по соседям соседей и т.д. Иногда такие модели называют сверточными энкодерами из-за схожести их архитектуры со свертками.

![Neighborhood aggregation and convolutional encoders](img/04.png)

# Сверточные энкодеры

![Алгоритм](img/05.png)

- Алгоритм строит эмбеддинги для нод рекурсивно 
- Вначале энкодер инициализируется атрибутами нод 
- Затем на каждой итерации агрегируются эмбеддинги соседей 
- Далее каждая нода получает новый эмбеддинг, скомбинированный из ее собственного эмбеддинга и агрегированного вектора.
- Затем все это отправляется в полносвязный слой и процесс повторяется K раз.

Эмбеддинг каждой ноды обогощается данными своих соседей, при этом размерность векторов остается постоянной во время всей процедуры. В результате мы получаем векторное представление графа.

Этот подход эксплуатируют:
- GNN (graph neural network)
- GCN (graph convolutional networks) \*
- column networks
- GraphSAGE

\* В лекциях cs224w понятия GNN и GCN сильно смешаны. Исторически сложилось, что GNN (graph neural network) - это отдельная история, развиваввшася самостоятельно и раньше. GNN - обобщает разные модели.

Разные подтипы модели определяются разными агрегирующей функцией (стр.4 алгоритма) и комбинирующей вектора функцией (стр.5)

### GCN
- агрегирующая функция - поэлементное среднее
- комбинируем взвешенной суммой 

### column network
- тоже самое, что и в GCN, только на выходе цикла стоит функция интерполяции, позволяющая сохранять локальную информацию в процессе итерации) [см. статью](https://arxiv.org/abs/1609.04508)

### GraphSAGE
- агрегирует среднее, макспуллинг или LSTM
- комбинируем конкатенацией

![GraphSAGE](img/09.png)


### 100500 производных - GAT, GIN и т.д.

### GNN

На мой взгляд объяснение GNN, которое дано в статье, просто обобщает все вышесказанное.

![GNN](img/07.png)

В базовой реализиации, алгоритм инциализируется случайным эмбеддингом для каждой ноды и на каждой итерации эмбеддинги накапливают информацию о своих соседях вот таким образом:

![GNN embeddings](img/08.png)

где вот эта $h$ - это произвольная дифференцируемая функция вида $рЖ \mathbb{R}^d \times \mathbb{R}^m \times \mathbb{R}^m \rightarrow \mathbb{R}^d$. Чтобы это не означало, на выходе после $K$ итераций мы должны получить вектор вида $\mathbf{z}_{v_i} = g({\mathbf{h}_i}^K)$, где $g$ - произвольня дифференцируемая функция вида $g: \mathbb{R}^d \rightarrow \mathbb{R}^d$, ну в общем ээто некая нейроночка, может быть MLP или какая-то другая конструкция. В статье приводится несколько ссылок на работы, посвященные этой тематике.

В лекции №8, после краткого рассказа с картинками, мы сразу перешли к объяснению GCN из-за чего произошла путаница.

![HNN-GCN](img/10.png)
![HNN-GCN](img/11.png)

\* Что там еще в статье? 
- мультимодальные графы (что делать с разными нодами и ребрами)
- структурные роли нод в графе
- эмбеддинги субграфов

[Representation Learning on Graphs: Methods and Applications](https://arxiv.org/abs/1709.05584)